In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from spx_history import *

- date range

In [ ]:
start_dt = dt.date(2021,1,1)
end_dt = dt.date.today()

In [ ]:
tu = pd.read_csv('data/spy_cst.csv')

In [ ]:
tickers = tu[tu['Ticker'] != '-']['Ticker'].unique().tolist()

- download data by date

In [ ]:
import requests
from io import StringIO

def download_file_by_date(query_date=dt.date(2025,1,1), filename="dividends", api_token=EODHD_API_KEY):
	df = pd.DataFrame()
	try:
		date_str = query_date.strftime("%Y-%m-%d")
		url = f'https://eodhd.com/api/eod-bulk-last-day/US?api_token={api_token}&date={date_str}&type={filename}&fmt=csv'
		data = requests.get(url)
		df = pd.read_csv(StringIO(data.content.decode()))

	except Exception as e:
		qt.log.warning(f"couldnt load {filename} data for date {query_date}")

	return df

In [ ]:
td = download_file_by_date(dt.date(2023, 12, 27), filename="dividends")
ts = download_file_by_date(dt.date(2023, 12, 27), filename="splits")

In [ ]:
dates = pd.date_range(start_dt, end_dt).date
filename = "dividends"
for d in dates:
	date_s = d.strftime("%Y-%m-%d")
	csv_fp = f'data_by_date/{filename}/{date_s}.csv'
	if Path(csv_fp).is_file():
		qt.log.info(f"{filename} file already exists for date : {d}")
		continue
	tf = download_file_by_date(query_date=d, filename=filename, api_token=EODHD_API_KEY)
	if len(tf) > 0:
		qt.log.info(f"saving {tf.shape} rows to {csv_fp}")
		tf.to_csv(csv_fp, index=False)

In [ ]:
dates = pd.date_range(start_dt, end_dt).date
filename = "splits"
for d in dates:
	date_s = d.strftime("%Y-%m-%d")
	csv_fp = f'data_by_date/{filename}/{date_s}.csv'
	if Path(csv_fp).is_file():
		qt.log.info(f"{filename} file already exists for date : {d}")
		continue
	tf = download_file_by_date(query_date=d, filename=filename, api_token=EODHD_API_KEY)
	if len(tf) > 0:
		qt.log.info(f"saving {tf.shape} rows to {csv_fp}")
		tf.to_csv(csv_fp, index=False)

- split data

In [ ]:
failed_tickers = download_data(get_dividends, tickers, start_dt, end_dt)

- split data

In [ ]:
failed_tickers_split = download_data(get_splits, tickers, start_dt, end_dt)

In [ ]:
failed_tickers_split

- intraday data

In [ ]:
failed_tickers_intraday = download_data(get_raw_intraday, tickers, start_dt, end_dt)